In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#
# Hopsitalization prediction model based on neural network trained 
# using hospitalization_neural_network_8.ipynb
#
class SNNHospitalization:
    #
    # Constructor
    #
    def __init__(self, weights=None):
        self.weights = weights
        self.label = ['patient_type']
        self.predictor = [
                'age',
                'sex',
                'pneumonia',
                'diabetes',
                'copd',
                'asthma',
                'inmsupr',
                'hypertension',
                'other_disease',
                'cardiovascular',
                'obesity',
                'renal_chronic',
                'tobacco'
            ]
        self.model = None
        self.scaler_age = None
        self.onehot = None
        self.metrics = [
              keras.metrics.TruePositives(name='tp'),
              keras.metrics.FalsePositives(name='fp'),
              keras.metrics.TrueNegatives(name='tn'),
              keras.metrics.FalseNegatives(name='fn'), 
              keras.metrics.BinaryAccuracy(name='accuracy'),
              keras.metrics.Precision(name='precision'),
              keras.metrics.Recall(name='recall'),
              keras.metrics.AUC(name='auc'),
        ]
        
    #
    # load model, weights and encoders from files previously generated by the training model
    #
    # Parameters:
    #    model_file: keras model in json format
    #    weights_file: keras trained model's weights in h5 format
    #    scaler_file: sklearn age encoder (StandardScaler) from trained model (pickle)
    #    onehot_file: sklearn categorical encoder (OneHotEncoder) from trained model (pickle)
    #    
    def load_snn_w_adam(self, model_file, weights_file, scaler_file, onehot_file):
        fread = open(model_file, "r")
        model_json = fread.read()
        fread.close()
        
        self.model = tf.keras.models.model_from_json(model_json)
        self.model.load_weights(weights_file)
        self.scaler_age = pickle.load(open(scaler_file, "rb"))
        self.onehot = pickle.load(open(onehot_file, "rb"))
        
    #
    # Predict needs for hospitalisation
    #
    # Parameters: 
    #    data: panda dataframe
    # Return value:
    #    predictions: numpy.ndarray
    #
    def predict(self, data):
        x = data[self.predictor].values
        y = data[self.label].values

        x_age = self.scaler_age.transform(x[0:, 0:1])
        x_remaining = self.onehot.transform(x[0:, 1:])

        X = np.concatenate((x_age, x_remaining), axis=1)            
        predictions = self.model.predict(x=X)

        return predictions # return numpy.ndarray
    
    #
    # Evaluate the test batch
    #
    # Parameters:
    #    data: panda dataframe
    def evaluate(self, data):
        # default values for Adam optimizer and evaluation
        batch_size=4096
        learning_rate = 1e-3
        beta_1 = 0.9
        beta_2=0.999
        epsilon=1e-07
        amsgrad=False
        
        x = data[self.predictor].values
        y = data[self.label].values
        
        x_age = self.scaler_age.transform(x[0:, 0:1])
        x_remaining = self.onehot.transform(x[0:, 1:])

        X = np.concatenate((x_age, x_remaining), axis=1)
        
        self.model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, 
                                                beta_1=beta_1,
                                                beta_2=beta_2,
                                                epsilon=epsilon,
                                                amsgrad=amsgrad), #'adam',
                    loss=keras.losses.BinaryCrossentropy(from_logits=False),
                    metrics=self.metrics)

        evaluation = self.model.evaluate(x=X, y=y, batch_size=batch_size, verbose=0)
        
        return evaluation
            
        

In [ ]:
#
#  Codes below are samples that show how to use SNNHospitalization class and visualize results
# 

In [ ]:
#
# Helper functions
#
import sklearn
from sklearn.metrics import confusion_matrix

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    print('No Hospitalization Correctly Detected (True Negatives): ', cm[0][0])
    print('Hospitalization Incorrectly Detected (False Positives): ', cm[0][1])
    print('No Hospitalization Missed (False Negatives): ', cm[1][0])
    print('Hospitalization Detected (True Positives): ', cm[1][1])
    print('Total Hospitalization: ', np.sum(cm[1]))

def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
#     plt.xlim([-0.5,20])
#     plt.ylim([80,100.5])
    plt.xlim([-0.5,100.5])
    plt.ylim([40,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
#
# Instantiate SNNHospitalization class, load previously trained models, load a sample batch data
# and finally, predict and evaluate the model
#
SOURCE = "../visualization/data/"

# Instantiate SNNHospitalization
snn = SNNHospitalization(weights=None)

# Load trained model from files. All files (parameters) are generated by *** hospitalization_neural_network_8.ipynb ***
snn.load_snn_w_adam(SOURCE + "snn_hospital.json", SOURCE + "snn_hospital.h5", SOURCE + "snn_hospital_age_scaler.pkl", SOURCE + "snn_hospital_encoder.pkl")

# Load a sample hospitalization dataset. It was generated by *** data_preprocessing.ipynb ***
df  = pd.read_csv(SOURCE + "hospitalization_sample_1.csv")

# Run the prediction
predictions = snn.predict(df)
y = df[snn.label].values

# Run the evaluation
evaluation = snn.evaluate(df)



In [ ]:
# Set color for graph
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Show ROC after evaluating this dataset: hospitalization_sample_1.csv
plot_roc("Train Weighted", y, predictions, color=colors[1])


In [ ]:
# Show evaluation results

for name, value in zip(snn.model.metrics_names, evaluation):
    print(name, ': ', value)
print()


In [ ]:
# Show confunsion matrics at 0.5 threshold

plot_cm(y, predictions, p=0.5)

In [ ]:
# Show confunsion matrics at 0.3 threshold

plot_cm(y, predictions, p=0.3)

In [ ]:
# Show prediction results and explain it in plain english

results=[]
for i, prediction in enumerate(predictions):
    results.append("This patient (row #{}) has a %.1f percent probability of being hospitalized due to COVID-19 as evaluated by this model.".format(i+1) % (100 * prediction[0]))

results

In [ ]:
################################################################################################################################